<a href="https://colab.research.google.com/github/Escandonn/IA-CAMBIO-VOZ/blob/main/Cambio_Voz_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Instalación de NeMo y Dependencias
###  En la primera celda de código de tu notebook en Colab, ejecuta los siguientes comandos para instalar NeMo y las dependencias necesarias:


In [ ]:

!pip install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html
BRANCH = 'r2.0.0rc0'
!pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]



# Importación de Bibliotecas y Configuración
###  En la siguiente celda, importa las bibliotecas necesarias y configura tu entorno. Asegúrate de ignorar las advertencias de preproducción para mantener limpio el flujo de salida:

In [ ]:

import warnings
warnings.filterwarnings('ignore')

import nemo
import nemo.collections.asr as nemo_asr
import nemo.collections.nlp as nemo_nlp
import nemo.collections.tts as nemo_tts
from IPython.display import Audio




# Descarga y Escucha del Audio de Ejemplo
### Descarga un archivo de audio de ejemplo para utilizarlo en tu aplicación. Puedes usar un archivo de muestra de la base de datos LibriSpeech Dev Clean:

In [ ]:

Audio_sample = '2086-149220-0033.wav'
!wget https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav


# Escucha el archivo de audio descargado para verificar su contenido:

In [ ]:
Audio(Audio_sample)


# Inicialización de Modelos Pre-entrenados
### Inicializa los modelos pre-entrenados que usarás para el reconocimiento de voz, procesamiento de texto y síntesis de voz. En las siguientes celdas, inicializa los modelos ASR, NLP y TTS:

In [ ]:
# Modelo de Reconocimiento de Voz - QuartzNet
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5").cuda()

# Modelo de Puntuación y Capitalización
punctuation = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained(model_name='punctuation_en_distilbert').cuda()

# Generador de Espectrogramas - FastPitch
spectrogram_generator = nemo_tts.models.FastPitchModel.from_pretrained(model_name="tts_en_fastpitch").cuda()

# Vocoder - HiFiGAN
vocoder = nemo_tts.models.HifiGanModel.from_pretrained(model_name="tts_en_hifigan").cuda()


# Procesamiento del Audio
### Transcribe el audio de ejemplo a texto, luego agrega puntuación y capitalización al texto transcritor. Después, genera un nuevo audio donde la voz dice una frase diferente:

In [ ]:
# Transcripción del audio de ejemplo
files = [Audio_sample]
raw_text = ''
text = ''
for fname, transcription in zip(files, quartznet.transcribe(audio=files)):
    raw_text = transcription

# Agregar puntuación y capitalización
res = punctuation.add_punctuation_capitalization(queries=[raw_text])
text = res[0]

# Frase nueva a decir
new_text = "Hola, ¿cómo estás?"

# Función para convertir texto a audio
def text_to_audio(text):
    parsed = spectrogram_generator.parse(text)
    spectrogram = spectrogram_generator.generate_spectrogram(tokens=parsed)
    audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)
    return audio.to('cpu').detach().numpy()

# Audio original
print(f'Texto reconocido sin procesar: {raw_text}.')
Audio(Audio_sample)


# Generación de Audio con la Nueva Frase
### Genera un nuevo audio utilizando la voz del audio original pero diciendo la nueva frase:

In [ ]:
# Convertir la nueva frase a audio
new_audio = text_to_audio(new_text)

# Escuchar el nuevo audio generado
print(f'Nueva frase: "{new_text}"')
Audio(new_audio, rate=22050)
